In [1]:
import pandas as pd
import requests

# Load your CSV file
#df = pd.read_csv('MS_City_Demand.csv')
pantry_df = pd.read_csv('MSPantryLatLong.csv')

# Function to get county name from latitude and longitude
def get_county(lat, long, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{long}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json()['results']
        for result in results:
            for component in result['address_components']:
                if 'administrative_area_level_2' in component['types']:
                    return component['long_name']
    return None

# Replace 'YOUR_API_KEY' with your actual Google Maps Geocoding API key
api_key = 'YOUR_API_KEY'

/Users/CameronField/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Apply the function to each row
#pantry_df['County'] = pantry_df.apply(lambda row: get_county(row['Latitude'], row['Longitude'], api_key), axis=1)
# Replace 'County' with an empty string in each cell of the 'County' column
#pantry_df['County'] = pantry_df['County'].str.replace(' County', '')
#pantry_df.to_csv('MSPantryLatLong.csv', index=False)
pantry_df.head()

,Name,Address,Latitude,Longitude,Capacity,County
0,Stewpot,"69 E Franklin Street, Natchez, MS",31.556106,-91.389928,10000,Adams
1,Rose Hill Baptist Church,"607 Madison Street, Natchez, MS",31.562803,-91.397244,500,Adams
2,Pilgrim Baptist Church,"117 Pilgrim Road, Natchez, MS",31.567402,-91.372177,500,Adams
3,First Assembly of God,"150 Liberty Street, Natchez, MS",31.552891,-91.382085,10000,Adams
4,61 South Church of Christ,"6 Beau Pre Road, Natchez, MS",31.482406,-91.368783,500,Adams


In [3]:
demand_df = pd.read_csv("MS_City_Demand.csv")
demand_df.head()

,City,County,FI Population 2021,2021 Cost Per Meal,Latitude,Longitude,CityWeight,2025 Cost Per Meal,FI Population 2025
0,Natchez,Adams,5530,3.35,31.5424,-91.3857,1.000000,3.79,3715
1,Corinth,Alcorn,5227,3.41,34.9484,-88.5136,0.834911,3.86,5127
2,Farmington,Alcorn,739,3.41,34.9226,-88.4461,0.117920,3.86,724
3,Glen,Alcorn,137,3.41,34.8582,-88.4176,0.021748,3.86,134
4,Rienzi,Alcorn,101,3.41,34.7633,-88.5340,0.016067,3.86,99


In [4]:
# Summing up the demand by county
demand_by_county = demand_df.groupby('County')['FI Population 2021'].sum().reset_index()

# Renaming the column
demand_by_county.rename(columns={'FI Population 2021': 'Demand'}, inplace=True)

# Suming up the capacity by county
capacity_by_county = pantry_df.groupby('County')['Capacity'].sum().reset_index()

Meals_Per_FI = 20
# Print the sum of 'Demand'
print("             Total Demand (FI People): ", demand_by_county['Demand'].sum())

# Print the sum of 'Capacity'
print("Total Capacity (FI People Equivalent): ", capacity_by_county['Capacity'].sum()/Meals_Per_FI)

             Total Demand (FI People):  451996
Total Capacity (FI People Equivalent):  204225.0


In [5]:


# Multiplying 'Demand' by Meals Per Person
demand_by_county['Demand'] = demand_by_county['Demand'] * Meals_Per_FI

# Suming up the capacity by county
capacity_by_county = pantry_df.groupby('County')['Capacity'].sum().reset_index()

# Merging the two dataframes on 'County'
df = pd.merge(demand_by_county, capacity_by_county, on='County')

# Creating a new column 'Percent Satisfied' which is 'Capacity' divided by 'Demand'
df['Percent Satisfied'] = (df['Capacity'] / df['Demand']) * 100
df["Meals Shortfall"] = (df['Demand'] - df['Capacity'])

# Sorting the dataframe by 'Demand' in descending order
df = df.sort_values('Demand', ascending=False)

# Display the dataframe
print(df)

        County  Demand  Capacity  Percent Satisfied  Meals Shortfall
17    Harrison  700440    460500          65.744389           239940
18       Hinds  634500    564000          88.888889            70500
21     Jackson  446640    200500          44.890740           246140
11      DeSoto  394060    190000          48.216008           204060
42      Rankin  353680    174500          49.338385           179180
12     Forrest  279820    120000          42.884712           159820
26  Lauderdale  253420    141000          55.638860           112420
32     Madison  225840    200500          88.779667            25340
23       Jones  222240     71000          31.947444           151240
28         Lee  221880    170500          76.843339            51380
25       Lamar  190240     50000          26.282590           140240
52  Washington  187660     40000          21.315144           147660
31     Lowndes  178020    101000          56.735198            77020
37   Oktibbeha  160420     71500  